In [3]:
# Byzantine Generals, N consensus using Qiskit, OOP Approach

# Import statements
import numpy as np
import copy, random, itertools
from qiskit import QuantumRegister, QuantumCircuit, execute
from qiskit import Aer
from qiskit.visualization import plot_histogram  # Visualizing stuff
from qiskit.quantum_info.analysis import hellinger_fidelity
from scipy.spatial.distance import sqeuclidean as distance  # define the distance/distribution used to unmask traitor

np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))


In [91]:
# Creating General Class for OOP Approach

class General():
    def __init__(self, general_number, is_commander=False):
        self.id = general_number
        self.is_commander = is_commander
        self.is_traitor = False
        self.states = None
        self.orders = None
        self.indices = None
        
    
    def __repr__(self):
        return f"General Number:{self.id}, Is Commander:{self.is_commander}, Is Traitor:{self.is_traitor}, Has States:{self.states is not None} \n"
    
    def receive_states(self, state_array):
        self.states = state_array[self.id, :]
    
    def receive_orders(self, order, indices):
        self.orders = order
        self.indices = indices
        # Some test of the commanders indices should be inserted here
        if self.is_traitor:
            self.indices = np.where(self.states == self.orders)[0]
            self.orders = not self.orders
            

class Byzantine_Problem():
    def __init__(self, N_gen, shots):
        self.N_gen = N_gen  # must be >= 2
        self.shots = shots
        self.backend = Aer.get_backend('qasm_simulator')  # The type of Qiskit simulator to use
        self.circuit = QuantumCircuit(self.N_gen+1)  # arg: number of qubits
        self.generals_list = []
        self.initial_state = None
        self.measurements = np.zeros((self.N_gen, self.shots), dtype=int)  # Initializing array with a row for each general, and space to store the corresponding orders 0/1/2 retreat, attack, neither
        self.populate()
        self.get_initial_state()
    
    def populate(self):
        self.generals_list.append(General(0, is_commander=True))
        for i in range(1,self.N_gen):
            self.generals_list.append(General(i, is_commander=False))
    
    def flip_allegiance(self, general_number):
        self.generals_list[general_number].is_traitor = not self.generals_list[general_number].is_traitor
    
    def get_initial_state(self):
            # Preparing the initial state as numpy array, values are amplitudes
        state = np.zeros(2**(self.N_gen+1))
        state[2**(self.N_gen-1)-1] = np.sqrt(1/3)  # Retreat order from commander
        state[2**(self.N_gen) + 2**(self.N_gen-1)] = np.sqrt(1/3)  # Attack order from commander
        for i in range(self.N_gen-1):
            state[2**(self.N_gen-1) + 2**i] = np.sqrt(1/(6*self.N_gen-6))  # commander '01'
            state[2**(self.N_gen) + 2**(self.N_gen-1) - 1 - 2**i] = np.sqrt(1/(6*self.N_gen-6))  # commander '10'
        self.initial_state = state
    
    def run_circuit(self, visualize=False):
        self.circuit.initialize(self.initial_state, self.circuit.qubits)   # Initialize the 5 qubits in the state to be distributed
        self.circuit.measure_active()  # Add a measurement of all 5 qubits in the logical (0/1) basis
        if visualize:
            display(self.circuit.draw(output='mpl'))
        # simulate the circuit and record the strings of the measured states in memory
        job = execute(self.circuit, backend=self.backend, shots=self.shots, memory=True, optimization_level=2)
        self.memory = job.result().get_memory(self.circuit)
    
    def distribute_states(self):
        # Converting into Array of corresponding orders
        v_dic = {'00': 0, '01': 2,  '10': 2,  '11' : 1, '0' :  0, '1' : 1}  # convert bitstrings to n_its and store them
        for i, bitstring in enumerate(self.memory):
            self.measurements[0][i] = v_dic[bitstring[0:2]] # Commander 2 indices
            for j in range(self.N_gen-1):
                self.measurements[j+1][i] = v_dic[bitstring[j+2]]
        for gen in self.generals_list:
            gen.receive_states(self.measurements)
    
    def verify_entanglement(self):
        """Need to add this step still."""
        pass
    
    def send_orders(self):
        if not self.generals_list[0].is_traitor:  # Loyal General Case
            order = bool(random.getrandbits(1))
            indices = np.where(self.measurements[0] == order)[0]
            for gen in self.generals_list:
                gen.receive_orders(order, indices)
        else:
            pass


    def realize_agreement(self):
        pairs = list(itertools.combinations(self.generals_list[1:], 2))
        for pair in pairs:
            play(pair)


# Creating my own Traitor Detection Game
def play(gen_pair, rounds=1000, threshold=0.01):
    failures = None
    # Check if they agree on the order then initiate game if they do not
    if gen_pair[0].orders != gen_pair[1].orders:
        failures = [0, 0]  # number of failures detected by player0 and player1 
        turn = bool(random.getrandbits(1))
        for i in range(rounds):
            if gen_pair[int(turn)].states[gen_pair[int(not turn)].indices[i]] != gen_pair[int(turn)].orders:
                failures[int(turn)] += 1
            if gen_pair[int(not turn)].states[gen_pair[int(turn)].indices[i]] != gen_pair[int(not turn)].orders:
                failures[int(not turn)] += 1
        if failures[0]/rounds > threshold:
            print(f"Lieutenant #{gen_pair[0].id} identifies Lieutenant #{gen_pair[1].id} as a traitor with failure rate {failures[0]/rounds}.")
        if failures[1]/rounds > threshold:
            print(f"Lieutenant #{gen_pair[1].id} identifies Lieutenant #{gen_pair[0].id} as a traitor with failure rate {failures[1]/rounds}.")
    # print(gen_pair[0].id, gen_pair[1].id, failures)
    





In [109]:
test = Byzantine_Problem(6,8000)
test.flip_allegiance(2)
test.flip_allegiance(3)
test.run_circuit()
test.distribute_states()
test.verify_entanglement()
test.send_orders()
test.realize_agreement()




Lieutenant #1 identifies Lieutenant #2 as a traitor with failure rate 0.129.
Lieutenant #1 identifies Lieutenant #3 as a traitor with failure rate 0.139.
Lieutenant #4 identifies Lieutenant #2 as a traitor with failure rate 0.127.
Lieutenant #5 identifies Lieutenant #2 as a traitor with failure rate 0.14.
Lieutenant #4 identifies Lieutenant #3 as a traitor with failure rate 0.137.
Lieutenant #5 identifies Lieutenant #3 as a traitor with failure rate 0.15.


In [ ]:
# Calculate failure rates in games for non traitor detecting traitor
# Allow traitorour commanders
# Allow traitors to selectively give out misinformation
# Add noise
# Add error correction